---
title: STEREO Magnetic field data pipeline
---

STEREO magnetic field is already in RTN coordinates, so no need to transform it.

Download data using `pyspedas`, but load it using `pycdfpp` (using `pyspedas` to load the data directly into `xarray` is very slow)

In [ ]:
#| export
from datetime import timedelta

import polars as pl
import pandas

from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

from typing import Iterable


In [ ]:
#| hide
#| default_exp pipelines/stereo/mag
%load_ext autoreload
%autoreload 2

## Loading data

In [ ]:
#| export
import os
os.environ['SPEDAS_DATA_DIR'] = f"{os.environ['HOME']}/data"
import pyspedas

In [ ]:
# | export
from ids_finder.utils.basic import cdf2pl, pmap

def download_data(
    start,
    end,
    probe: str = "a",
    datatype = None,
) -> Iterable[str]:
    "List of CDF files"
    trange = [start, end]
    files = pyspedas.stereo.mag(trange, probe=probe, datatype=datatype, downloadonly=True)
    return files


def load_data(
    start,
    end,
    datatype = None,
    probe: str = "a",
):
    data = download_data(start, end, probe, datatype)
    return pl.concat(data | pmap(cdf2pl, var_name="BFIELD"))


## Preprocessing data

In [ ]:
#| export
from ids_finder.utils.basic import resample
from pipe import select

In [ ]:
# | export
def preprocess_data(
    raw_data: pl.LazyFrame,
    ts: str = "1s",  # time resolution
) -> pl.DataFrame:
    """
    Preprocess the raw dataset (only minor transformations)

    - Downsample the data to a given time resolution
    - Applying naming conventions for columns
    """
    every = pandas.Timedelta(ts)
    period = 2 * every

    return (
        raw_data.pipe(resample, every=every, period=period)
        .rename(
            {
                "BFIELD_0": "b_r",
                "BFIELD_1": "b_t",
                "BFIELD_2": "b_n",
                "BFIELD_3": "b_mag",
            }
        )
        .collect()
    )

## Processing data

In [ ]:
#| export
from ids_finder.utils.basic import partition_data_by_year

In [ ]:
#| export
def process_data(
    raw_data: pl.DataFrame,
    ts: str = None,  # time resolution
    coord: str = None,
) -> dict[str, pl.DataFrame]:
    """
    Corresponding to primary data layer, where source data models are transformed into domain data models

    - Partitioning data, for the sake of memory
    """
    return partition_data_by_year(raw_data)

## Pipeline

In [ ]:
# | export
from ids_finder.core.pipeline import extract_features
from ids_finder.pipelines.default.data_mag import create_pipeline_template


def create_pipeline(sat_id="STA", source="MAG"):
    return create_pipeline_template(
        sat_id=sat_id,
        source=source,
        load_data_fn=load_data,
        preprocess_data_fn=preprocess_data,
        process_data_fn=process_data,
        extract_features_fn=extract_features,
    )

## Obsolete codes

NOTE: one can also use `speasy` to download data, however this is slower for `STEREO` data.

In [ ]:
%%markdown
sat_fgm_product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD
sat_fgm_product = 'cda/STA_L1_MAG_RTN/BFIELD'
products = [sat_fgm_product]

dataset = spz.get_data(products, test_trange, disable_proxy=True)
sat_fgm_data  = dataset[0]
data_preview(sat_fgm_data)

Download data in a background thread

In [ ]:
%%markdown

@threaded
def download_data(products, trange):
    logger.info("Downloading data")
    spz.get_data(products, trange, disable_proxy=True)
    logger.info("Data downloaded")
    
download_data(products, trange)

In [ ]:
import speasy as spz

In [ ]:
cda_tree: spz.SpeasyIndex = spz.inventories.tree.cda
product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN

logger.info(product.description)
logger.info(product.ID)
logger.info(product.BFIELD.CATDESC)
logger.info(product.BFIELD.spz_uid())

# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.
# spz.inventories.data_tree.cda.STEREO.STEREOA.IMPACT_MAG.STA_LB_MAG_RTN.description
# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.MAGFLAGUC.CATDESC
spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD.CATDESC
# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD.